# Predicit store sales using XGBoost Model

### Call Libraries

In [1]:
# 1.0 Call libraries
%reset -f
# 1.1 For data manipulations
import numpy as np
import pandas as pd
from numpy.random import default_rng

# 1.2 For plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl     # For creating colormaps

# 1.3 For data processing
from sklearn.preprocessing import StandardScaler

# 1.4 OS related
import os
import time
# 1.5 for working in ipython
#%matplotlib qt5
#%matplotlib inline

from scipy.stats import scoreatpercentile, pearsonr

from sklearn.preprocessing import StandardScaler as ss

# 1.3 Dimensionality reduction and noise removal
from sklearn.decomposition import PCA
from xgboost import XGBRegressor

# 1.5 Model pipelining
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# 1.6 Hyperparameter optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Install packages

In [4]:
# 0.0
# For skopt routines
! pip install scikit-optimize

# 0.1 For plotting skopt results
! pip install 'scikit-optimize[plots]'

In [5]:
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer

### Mount Drive and read data

In [6]:
# Mount gdrive
#from google.colab import drive
#drive.mount('/gdrive')
#path = "/gdrive/MyDrive/GoogleCollabDataFiles/"
#os.chdir(path)
wmart = pd.read_csv("/kaggle/input/walmart-sales-dataset-of-45stores/walmart-sales-dataset-of-45stores.csv",
                    parse_dates = ['Date'] 
                )


In [7]:
wmart.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [8]:
wmart.dtypes

Store                    int64
Date            datetime64[ns]
Weekly_Sales           float64
Holiday_Flag             int64
Temperature            float64
Fuel_Price             float64
CPI                    float64
Unemployment           float64
dtype: object

In [9]:
wmart.drop(['Date'], axis=1, inplace=True)

In [10]:
wmart.dtypes

Store             int64
Weekly_Sales    float64
Holiday_Flag      int64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [11]:
y=wmart.pop('Weekly_Sales')

In [12]:
y = np.log(y)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(wmart,
                                                    y,
                                                    test_size=0.35,
                                                    shuffle = True
                                                    )

### Using Gridsearch to identify the best hyperparameters

In [14]:
steps_xg = [('sts', ss() ),
            ('pca', PCA()),
            ('xg',  XGBRegressor(silent = False,
                                  n_jobs=3,objective="reg:squarederror",verbosity=0)        # Specify other parameters here
            )
            ]

pipe_xg = Pipeline(steps_xg)

In [15]:
parameters = {'xg__learning_rate':  [0.03, 0.05], # learning rate decides what percentage
                                                  #  of error is to be fitted by
                                                  #   by next boosted tree.
                                                  # See this answer in stackoverflow:
                                                  # https://stats.stackexchange.com/questions/354484/why-does-xgboost-have-a-learning-rate
                                                  # Coefficients of boosted trees decide,
                                                  #  in the overall model or scheme, how much importance
                                                  #   each boosted tree shall have. Values of these
                                                  #    Coefficients are calculated by modeling
                                                  #     algorithm and unlike learning rate are
                                                  #      not hyperparameters. These Coefficients
                                                  #       get adjusted by l1 and l2 parameters
              'xg__n_estimators':   [1500,2000,2500],  # Number of boosted trees to fit
                                                  # l1 and l2 specifications will change
                                                  # the values of coeff of boosted trees
                                                  # but not their numbers

              'xg__max_depth':      [5,7,9],
              'pca__n_components' : [0.95]
              }     

In [16]:
clf = GridSearchCV(pipe_xg,            # pipeline object
                   parameters,         # possible parameters
                   n_jobs = 4,         # USe parallel cpu threads
                   cv =5 ,             # No of folds
                   verbose =0,         # Higher the value, more the verbosity
                   scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error','max_error'],
                   refit='neg_mean_squared_error'
                   )

In [17]:
print("\n\n--Takes time...---\n")
start = time.time()
clf.fit(X_train, y_train)
end = time.time()
print()
(end - start)/60     



--Takes time...---



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('sts', StandardScaler()),
                                       ('pca', PCA()),
                                       ('xg',
                                        XGBRegressor(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                                                     colsample_bytree=None,
                                                     early_stopping_rounds=None,
                                                     enable_categorical=False,
                                                     eval_metric=None,
                                                     gamma=None, gpu_id=None,
                                                     grow_policy=None,

18.92771189212799

In [18]:

f"Best score: {clf.best_score_} "            


print()
f"Best parameter set {clf.best_params_}"

'Best score: -0.07643511732649547 '

"Best parameter set {'pca__n_components': 0.95, 'xg__learning_rate': 0.05, 'xg__max_depth': 9, 'xg__n_estimators': 2000}"

In [19]:
y_pred = clf.predict(X_test)
print("--Few predictions--\n")
max(np.exp(y_pred[:]))


--Few predictions--



2995250.5

In [20]:
model = XGBRegressor(pca__n_components= 0.95,
                     xg__learning_rate=0.05, 
                     xg__max_depth= 9, 
                     xg__n_estimators=2000,
                     objective='reg:squarederror')
					
model.fit(X_train, y_train)
ypred = model.predict(X_test)

mse = mean_squared_error(y_test, ypred)

print("\n MSE: %.2f" % mse)
print("\n RMSE: %.2f" % (mse**(1/2.0)))

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, pca__n_components=0.95, predictor='auto',
             random_state=0, reg_alpha=0, ...)


 MSE: 0.01

 RMSE: 0.09


### Using Random Search CV for identifying the best hyperparameters

In [21]:
rs = RandomizedSearchCV(
                          pipe_xg,
                          param_distributions=parameters,
                          scoring= ['neg_mean_absolute_error', 'neg_mean_squared_error','max_error'],
                          n_iter=10,           # Max combination of
                                              # parameter to try. Default = 10
                          verbose = 1,
                          refit = 'neg_mean_squared_error',
                          n_jobs = 3,          # Use parallel cpu threads
                          cv = 10               # No of folds.
                                              # So n_iter * cv combinations
                        )


In [22]:
start = time.time()
rs.fit(X_train, y_train)
end = time.time()
print()
(end - start)/60   # 4 minutes

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('sts', StandardScaler()),
                                             ('pca', PCA()),
                                             ('xg',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=None,
                                                           early_stopping_rounds=None,
                                                           enable_categorical=False,
                                                           eval_metric=None,
                                                           gamma=None

23.874944587548573

In [23]:
f"Best score: {clf.best_score_} "            

print()
f"Best parameter set {clf.best_params_}"

'Best score: -0.07643511732649547 '

"Best parameter set {'pca__n_components': 0.95, 'xg__learning_rate': 0.05, 'xg__max_depth': 9, 'xg__n_estimators': 2000}"

In [24]:
y_pred = clf.predict(X_test)
print("--Few predictions--\n")
max(np.exp(y_pred[:]))


--Few predictions--



2995250.5

In [25]:
model = XGBRegressor(pca__n_components= 0.95, 
                     xg__learning_rate=0.05, 
                     xg__max_depth= 9, 
                     xg__n_estimators=2000,
                     objective='reg:squarederror')
model.fit(X_train, y_train)
ypred = model.predict(X_test)

mse = mean_squared_error(y_test, ypred)

print("\n MSE: %.2f" % mse)
print("\n RMSE: %.2f" % (mse**(1/2.0)))

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, pca__n_components=0.95, predictor='auto',
             random_state=0, reg_alpha=0, ...)


 MSE: 0.01

 RMSE: 0.09


### Using Bayesian Search to identify best hypermeters

In [26]:
params={
         'min_child_weight': Integer(0, 50),  # min no of instances needed to be in each node before split
         'max_depth': Integer(1, 10),         # Maximum depth of a tree.
         'subsample': Real (0.5, 1.0, 'uniform'),  # Subsample ratio of the training instances. 
                                                   # Setting it to 0.5 means that XGBoost would 
                                                   # randomly sample half of the training data prior
                                                   # to growing trees. and this will prevent overfitting. 
         'colsample_bytree': Real(0.5, 1.0, 'uniform'), # subsample ratio of cols when constructing each tree
         'reg_lambda':Real(1e-5,100,prior = 'log-uniform'), # L2 reg term on weights. Increasing this value will make model more conservative
                                                            #  ie move towards simpler model
         'reg_alpha': Real(1e-5,100,prior= 'log-uniform'),  # L1 reg
         'gamma': Real(1e-9, 0.5, 'log-uniform'),   # Min loss reduction required to make a further partition
                                                    # on a leaf node of the tree. The larger gamma is, the 
                                                    # more conservative the algorithm will be leading to simpler models.
         'learning-rate': Real(0.01,0.2,prior='log-uniform'), # How much weight should shrink after each step 
                                                              # to make boosting process (NOT model) more conservative
         'scale_pos_weight': Real(1e-6, 500, 'log-uniform'),  # Control the balance of pos and neg weights, useful for unbalanced classes
         'n_estimators': Integer(100, 3000)   # Max number of trees
        }

In [27]:
bayes_tuner=BayesSearchCV(
                            XGBRegressor(
                                            n_jobs = 4,
                                            objective='reg:squarederror',
                                            eval_metric = 'rmse',
                                            silent=1,
                                            tree_method='approx'
                                            ),
                            params,
                            n_iter=5,              # No of parameter settings that are tried 
                            scoring='neg_mean_squared_error',     # The criteria for best model. Maximise
                            optimizer_kwargs={     # Dict of arguments passed to Optimizer.
                                               'n_initial_points' : 5,  # Initial random pts before intelligence sets in
                                                                        #  default: 10
                                               'initial_point_generator':'random' # Random serach for init pts
                                              }, 
                            cv = KFold(
                                                  n_splits=10,
                                                  shuffle=True,
                                                  random_state=42
                                                ),
                            refit = True, # Refit the best estimator
                                          #   so as to make predictions
                            verbose = 1,
                            return_train_score= True
                    )


In [28]:
start = time.time()
res=bayes_tuner.fit(
                      X_train,
                      y_train,
                      #callback=status_print
                    )

end = time.time()

f"{(end - start)/60} minutes "    # 3 minutes on Colab

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


'3.193155868848165 minutes '

In [29]:
res.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.7822743471804519, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse',
             gamma=3.396615561557749e-05, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning-rate=0.012169796606039613, learning_rate=0.300000012,
             max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=7,
             max_leaves=0, min_child_weight=10, missing=nan,
             monotone_constraints='()', n_estimators=1609, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.005479296448630956, ...)

In [30]:
res.best_score_

-0.0074286002978073435

In [31]:
res.best_params_

OrderedDict([('colsample_bytree', 0.7822743471804519),
             ('gamma', 3.396615561557749e-05),
             ('learning-rate', 0.012169796606039613),
             ('max_depth', 7),
             ('min_child_weight', 10),
             ('n_estimators', 1609),
             ('reg_alpha', 0.005479296448630956),
             ('reg_lambda', 0.01359256878101883),
             ('scale_pos_weight', 0.08260970994852401),
             ('subsample', 0.919386323228355)])

In [32]:
model = XGBRegressor(colsample_bytree=0.7289138671356226, 
                     eval_metric='rmse',
                    gamma=3.450759283378757e-05, 
                    learning_rate=0.01593327330352174,
                    max_depth=7, 
                    min_child_weight=47, 
                    n_estimators=1620, 
                    objective='reg:squarederror', 
                    reg_alpha=0.003168973388873482,
                    reg_lambda=5.785085131033634e-05,
                    scale_pos_weight=51.351027196303, 
                    subsample=0.9379088882197723, 
                    )

In [33]:
model.fit(X_train, y_train)
ypred = model.predict(X_test)
mse = mean_squared_error(y_test, ypred)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1,
             colsample_bytree=0.7289138671356226, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse',
             gamma=3.450759283378757e-05, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.01593327330352174, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=7, max_leaves=0,
             min_child_weight=47, missing=nan, monotone_constraints='()',
             n_estimators=1620, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0.003168973388873482,
             reg_lambda=5.785085131033634e-05, ...)

In [34]:
print("MSE: %.2f" % mse)
print("RMSE: %.2f" % (mse**(1/2.0)))

MSE: 0.01
RMSE: 0.10
